<a href="https://colab.research.google.com/github/manjotmb20/fastai-v3/blob/master/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA

dataset=pd.read_csv('train.csv')

dataset2=pd.read_csv('test.csv')
dataset=dataset.fillna(dataset.mean())
dataset2=dataset2.fillna(dataset2.mean())
embarked=pd.get_dummies(dataset.Embarked,prefix='Embarked')
embarked2=pd.get_dummies(dataset2.Embarked,prefix='Embarked')
dataset=pd.concat([dataset,embarked],axis=1)
dataset2=pd.concat([dataset2,embarked2],axis=1)
dataset.drop(['Cabin', 'Ticket','Embarked','Name','PassengerId'],axis=1, inplace=True)
dataset2.drop(['Cabin', 'Ticket','Embarked','Name','PassengerId'],axis=1, inplace=True)
lc=LabelEncoder()
dataset[['Sex']]=lc.fit_transform(dataset[['Sex']])
dataset2[['Sex']]=lc.transform(dataset2[['Sex']])
sc=StandardScaler()
dataset[['Fare']]=sc.fit_transform(dataset[['Fare']])
dataset2[['Fare']]=sc.transform(dataset2[['Fare']])
sc1=StandardScaler()
dataset[['Age']]=sc1.fit_transform(dataset[['Age']])
dataset2[['Age']]=sc1.transform(dataset2[['Age']])
df_test=dataset['Survived']
df_train=dataset.drop('Survived',axis=1)
test=dataset2

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score,accuracy_score
x_train,x_val,y_train,y_val=train_test_split(df_train,df_test,test_size=0.2,random_state=9)
classifier=LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)
ypred=classifier.predict(x_val)
print(roc_auc_score(y_val,ypred))
print(accuracy_score(y_val,ypred))
df=pd.DataFrame()
df['pred']=ypred

classifier1=DecisionTreeClassifier()
classifier1.fit(x_train,y_train)
ypred1=classifier1.predict(x_val)
df['pred2']=ypred1
print(roc_auc_score(y_val,ypred1))
print(accuracy_score(y_val,ypred1))

survived=pd.get_dummies(df.pred,prefix='Survived')
survived_classifier2=pd.get_dummies(df.pred2,prefix='Survsecond')
df=pd.concat([df,survived],axis=1)
df=pd.concat([df,survived_classifier2],axis=1)
df.drop(['pred','pred2'],axis=1,inplace=True)

0.7627848101265823
0.776536312849162
0.7264556962025317
0.7374301675977654


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [3]:
type(survived)

pandas.core.frame.DataFrame

In [4]:
df

,Survived_0,Survived_1,Survsecond_0,Survsecond_1
0,1,0,1,0
1,1,0,1,0
2,1,0,1,0
3,1,0,1,0
4,0,1,0,1
...,...,...,...,...
174,0,1,0,1
175,0,1,0,1
176,1,0,1,0
177,0,1,0,1


In [0]:
mem_mat=df

In [0]:
cor_mat=mem_mat.dot(mem_mat.T)

In [7]:
cor_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178
0,2,2,2,2,0,2,1,2,1,2,2,1,2,2,0,1,1,0,0,2,2,2,1,2,2,1,2,2,2,0,2,2,0,1,1,2,0,2,2,1,...,1,2,1,1,2,2,2,2,1,1,2,2,1,2,2,1,2,1,0,1,2,0,2,0,2,1,2,0,0,2,2,2,2,0,1,0,0,2,0,0
1,2,2,2,2,0,2,1,2,1,2,2,1,2,2,0,1,1,0,0,2,2,2,1,2,2,1,2,2,2,0,2,2,0,1,1,2,0,2,2,1,...,1,2,1,1,2,2,2,2,1,1,2,2,1,2,2,1,2,1,0,1,2,0,2,0,2,1,2,0,0,2,2,2,2,0,1,0,0,2,0,0
2,2,2,2,2,0,2,1,2,1,2,2,1,2,2,0,1,1,0,0,2,2,2,1,2,2,1,2,2,2,0,2,2,0,1,1,2,0,2,2,1,...,1,2,1,1,2,2,2,2,1,1,2,2,1,2,2,1,2,1,0,1,2,0,2,0,2,1,2,0,0,2,2,2,2,0,1,0,0,2,0,0
3,2,2,2,2,0,2,1,2,1,2,2,1,2,2,0,1,1,0,0,2,2,2,1,2,2,1,2,2,2,0,2,2,0,1,1,2,0,2,2,1,...,1,2,1,1,2,2,2,2,1,1,2,2,1,2,2,1,2,1,0,1,2,0,2,0,2,1,2,0,0,2,2,2,2,0,1,0,0,2,0,0
4,0,0,0,0,2,0,1,0,1,0,0,1,0,0,2,1,1,2,2,0,0,0,1,0,0,1,0,0,0,2,0,0,2,1,1,0,2,0,0,1,...,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,2,1,0,2,0,2,0,1,0,2,2,0,0,0,0,2,1,2,2,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,0,0,0,0,2,0,1,0,1,0,0,1,0,0,2,1,1,2,2,0,0,0,1,0,0,1,0,0,0,2,0,0,2,1,1,0,2,0,0,1,...,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,2,1,0,2,0,2,0,1,0,2,2,0,0,0,0,2,1,2,2,0,2,2
175,0,0,0,0,2,0,1,0,1,0,0,1,0,0,2,1,1,2,2,0,0,0,1,0,0,1,0,0,0,2,0,0,2,1,1,0,2,0,0,1,...,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,2,1,0,2,0,2,0,1,0,2,2,0,0,0,0,2,1,2,2,0,2,2
176,2,2,2,2,0,2,1,2,1,2,2,1,2,2,0,1,1,0,0,2,2,2,1,2,2,1,2,2,2,0,2,2,0,1,1,2,0,2,2,1,...,1,2,1,1,2,2,2,2,1,1,2,2,1,2,2,1,2,1,0,1,2,0,2,0,2,1,2,0,0,2,2,2,2,0,1,0,0,2,0,0
177,0,0,0,0,2,0,1,0,1,0,0,1,0,0,2,1,1,2,2,0,0,0,1,0,0,1,0,0,0,2,0,0,2,1,1,0,2,0,0,1,...,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,2,1,0,2,0,2,0,1,0,2,2,0,0,0,0,2,1,2,2,0,2,2


In [0]:
y_prob=classifier.predict_proba(x_val)
y_prob2=classifier1.predict_proba(x_val)

In [0]:
objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob2,columns=['Survived','NotSurvived'])

In [0]:
gpclass_mat=pd.DataFrame(columns=['Survived','NotSurvived'],index=['class1surv','class1notsurv','class2surv','class2notsurv'])

In [11]:
prob1s=np.sum(objclass_mat1.Survived)/len(objclass_mat1)
prob1ns=np.sum(objclass_mat1.NotSurvived)/len(objclass_mat1)
prob2s=np.sum(objclass_mat2.Survived)/len(objclass_mat2)
prob2ns=np.sum(objclass_mat2.NotSurvived)/len(objclass_mat2)
prob1s

0.6270649150490318

In [0]:
list1=[]
list1.append(prob1s)
list1.append(prob1ns)
list1.append(prob2s)
list1.append(prob2ns)

list2=[]
list2.append(prob1ns)
list2.append(prob1s)
list2.append(prob2ns)
list2.append(prob2s)
gpclass_mat['Survived']=list1
gpclass_mat['NotSurvived']=list2



In [13]:
gpclass_mat

,Survived,NotSurvived
class1surv,0.627065,0.372935
class1notsurv,0.372935,0.627065
class2surv,0.615270,0.384730
class2notsurv,0.384730,0.615270


In [0]:
kmshape=mem_mat.shape
kcshape=cor_mat.shape

In [15]:
kcshape

(179, 179)

In [0]:
Km=np.zeros(shape=kmshape)
Kc=np.zeros(shape=kcshape)
Khm=mem_mat
Khc=cor_mat

In [0]:
import sys
threshold=sys.float_info.epsilon

In [18]:
Km=np.zeros(shape=kmshape)
Kc=np.zeros(shape=kcshape)
Khm=np.array(mem_mat)
Khc=np.array(cor_mat)
while LA.norm((Km-Khm),'fro')>threshold:
  Km=Khm
  d=np.zeros(shape=(kmshape[0]))
  for i in range(kmshape[0]):
    d[i]=np.sum(Km[i],axis=0)
  Km=Km/d.reshape((d.shape[0],1))  



KeyboardInterrupt: ignored

In [185]:
d.shape

(179,)

In [196]:
a=np.empty(shape=(2,4))
a[0][1]=0.71
a[1][1]=0.2
c=np.zeros(shape=(2))
for i in range(2):
  c[i]=np.sum(a[i],axis=0)
for i in range(a.shape[0]):
  for j in range(a.shape[1]):
    a[i][j]=a[j][i]= np.sqrt(a[i][j]*a[j][i]) 
print(c)
print(a)
final=a/c.reshape((c.shape[0],1))
print(final)

IndexError: ignored

In [0]:
c